## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract findings in radiology reports](https://aws.amazon.com/marketplace/pp/prodview-kriuw433xdo4u)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

### Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline

Overview:

The Radiology Entity Detection, Assertion Status Assignment, and Relation Extraction Pipeline is a comprehensive solution designed to enhance the analysis of radiology documents. Its primary focus lies in identifying crucial entities within radiology texts, assigning assertion statuses to these entities, and extracting relevant relations among them.


- **Model**: `en.explain_doc.clinical_radiology.pipeline`
- **Model Description**: A pipeline for detecting posology entities with the `ner_radiology` NER model, assigning their assertion status with `assertion_dl_radiology` model, and extracting relations between posology-related terminology with `re_test_problem_finding` relation extraction model.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "en-explain-doc-clinical-radiology-pipeline"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

--------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    """Cardiac size cannot be evaluated. Large left pleural effusion is new. Small right effusion is new. The upper lungs are clear. Right lower lobe opacities are better seen in prior CT. There is no pneumothorax. There are mild degenerative changes in the thoracic spine""",

    """Bilateral breast ultrasound was subsequently performed, which demonstrated an ovoid mass measuring approximately 0.5 x 0.5 x 0.4 cm in diameter located within the anteromedial aspect of the left shoulder. This mass demonstrates isoechoic echotexture to the adjacent muscle, with no evidence of internal color flow. This may represent benign fibrous tissue or a lipoma."""
]

sample_text = """Abdominal ultrasound shows a small simple cyst in the left kidney, measuring 1.2 cm. The liver and gallbladder are unremarkable. No signs of cholelithiasis or biliary dilation."""

### JSON

In [9]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [10]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,Abdominal ultrasound,0,19,Imaging_Test,0.95295
1,small simple cyst,29,45,ImagingFindings,0.6244
2,left kidney,54,64,BodyPart,0.72424996
3,1.2 cm,77,82,Measurements,0.9805
4,liver,89,93,BodyPart,0.8004
5,gallbladder,99,109,BodyPart,0.7609
6,unremarkable,115,126,ImagingFindings,0.7926
7,cholelithiasis,141,154,Disease_Syndrome_Disorder,0.7591
8,biliary dilation,159,174,Symptom,0.4174


In [11]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,small simple cyst,29,45,ImagingFindings,Confirmed,0.9981
1,unremarkable,115,126,ImagingFindings,Confirmed,0.992
2,cholelithiasis,141,154,Disease_Syndrome_Disorder,Negative,0.9977
3,biliary dilation,159,174,Symptom,Negative,0.9828


In [12]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,Abdominal ultrasound,0,19,Imaging_Test,small simple cyst,29,45,ImagingFindings,is_related,1.0
1,Abdominal ultrasound,0,19,Imaging_Test,left kidney,54,64,BodyPart,is_related,0.99999547
2,small simple cyst,29,45,ImagingFindings,left kidney,54,64,BodyPart,is_related,0.94614494
3,small simple cyst,29,45,ImagingFindings,1.2 cm,77,82,Measurements,is_related,0.9952284
4,liver,89,93,BodyPart,unremarkable,115,126,ImagingFindings,not_related,0.9999999


### JSON Lines

In [13]:
import json

def create_jsonl(records):
    
    if isinstance(records, str):
        records = [records]
    
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "Abdominal ultrasound", "begin": 0, "end": 19, "ner_label": "Imaging_Test", "ner_confidence": "0.95295"}, {"ner_chunk": "small simple cyst", "begin": 29, "end": 45, "ner_label": "ImagingFindings", "ner_confidence": "0.6244"}, {"ner_chunk": "left kidney", "begin": 54, "end": 64, "ner_label": "BodyPart", "ner_confidence": "0.72424996"}, {"ner_chunk": "1.2 cm", "begin": 77, "end": 82, "ner_label": "Measurements", "ner_confidence": "0.9805"}, {"ner_chunk": "liver", "begin": 89, "end": 93, "ner_label": "BodyPart", "ner_confidence": "0.8004"}, {"ner_chunk": "gallbladder", "begin": 99, "end": 109, "ner_label": "BodyPart", "ner_confidence": "0.7609"}, {"ner_chunk": "unremarkable", "begin": 115, "end": 126, "ner_label": "ImagingFindings", "ner_confidence": "0.7926"}, {"ner_chunk": "cholelithiasis", "begin": 141, "end": 154, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.7591"}, {"ner_chunk": "biliary dilation", "begin": 159, "end": 174, "ner_la

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [17]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    display(data)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [20]:
process_s3_json_output_and_save(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': 'Cardiac size',
    'begin': 0,
    'end': 11,
    'ner_label': 'ImagingFindings',
    'ner_confidence': '0.8463'},
   {'ner_chunk': 'left',
    'begin': 40,
    'end': 43,
    'ner_label': 'Direction',
    'ner_confidence': '0.9433'},
   {'ner_chunk': 'pleural effusion',
    'begin': 45,
    'end': 60,
    'ner_label': 'Disease_Syndrome_Disorder',
    'ner_confidence': '0.69865'},
   {'ner_chunk': 'right',
    'begin': 76,
    'end': 80,
    'ner_label': 'Direction',
    'ner_confidence': '0.9554'},
   {'ner_chunk': 'effusion',
    'begin': 82,
    'end': 89,
    'ner_label': 'ImagingFindings',
    'ner_confidence': '0.9442'},
   {'ner_chunk': 'upper lungs',
    'begin': 103,
    'end': 113,
    'ner_label': 'BodyPart',
    'ner_confidence': '0.5999'},
   {'ner_chunk': 'clear',
    'begin': 119,
    'end': 123,
    'ner_label': 'ImagingFindings',
    'ner_confidence': '0.7488'},
   {'ner_chunk': 'Right',
    'begin': 126,
    'end': 130,
    'ner_la

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [23]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "Cardiac size", "begin": 0, "end": 11, "ner_label": "ImagingFindings", "ner_confidence": "0.8463"}, {"ner_chunk": "left", "begin": 40, "end": 43, "ner_label": "Direction", "ner_confidence": "0.9433"}, {"ner_chunk": "pleural effusion", "begin": 45, "end": 60, "ner_label": "Disease_Syndrome_Disorder", "ner_confidence": "0.69865"}, {"ner_chunk": "right", "begin": 76, "end": 80, "ner_label": "Direction", "ner_confidence": "0.9554"}, {"ner_chunk": "effusion", "begin": 82, "end": 89, "ner_label": "ImagingFindings", "ner_confidence": "0.9442"}, {"ner_chunk": "upper lungs", "begin": 103, "end": 113, "ner_label": "BodyPart", "ner_confidence": "0.5999"}, {"ner_chunk": "clear", "begin": 119, "end": 123, "ner_label": "ImagingFindings", "ner_confidence": "0.7488"}, {"ner_chunk": "Right", "begin": 126, "end": 130, "ner_label": "Direction", "ner_confidence": "0.9839"}, {"ner_chunk": "lower lobe opacities", "begin": 132, "end": 151, "ner_label": "Symptom", "ner_confi

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-clinical-radiology-pipel-2024-08-21-11-07-13-857


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

